# Entregable 1

## Sección 1. Entendimiento del negocio y enfoque analítico.

**Oportunidad/problema negocio:** Con el modelo se solucionará el problema de lo costoso que es hacer las clasificaciones de las opiniones al automatizarse, al mismo tiempo, se podrá obtener información sobre las necesidades de los habitantes y en que ODS se deben focalizar esfuerzos para satisfacer sus necesidades.

**Objetivos y criterios de éxito desde el punto de vista
del negocio:** Se quiere disminuir la pobreza, mejorar la educación, igualdad de género (ODS 3,4 Y 5) y dar calidad de vida a las personas en general. Los criterios de éxito establecerán que se cumplieron los objetivos si se identificaron correctamente los ODS a priorizar y si se le dió un beneficio proporcional a las poblaciones.

**Organización y rol dentro de ella que se beneficia con
la oportunidad definida:**
La UNFPA se beneficiará del software resultante del proyecto, principalmente el área de la empresa que juega el rol financiar y ejecutar proyectos para cumplir los ODF

**Impacto que puede tener en Colombia este
proyecto:** Este proyecto puede beneficiar la calidad de vida de muchas personas de bajos recursos en colombia, mejorar su educación y promover la igualdad de género.

**Enfoque analítico. Descripción de la categoría de
análisis (descriptivo, predictivo, etc.) , tipo y tarea de
aprendizaje e incluya las técnicas y algoritmos que
propone utilizar:** El modelo estará en la categoría de análisis descriptivo, se planean utilizar los algoritmos de Naive Bayes, KNN y regresión logística


## Sección 2. Entendimiento y preparación de los datos.

### Importaciones

In [155]:
import numpy as np
import pandas as pd
import re

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import spacy
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import TransformedTargetRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


import scipy.stats as stats
import statsmodels.api as sm

import joblib

!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 56.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Carga de datos

In [156]:
df_unfpa = pd.read_excel('/content/ODScat_345.xlsx', index_col=0)
df_unfpa.shape

(4049, 1)

### Exploración y descripción de los datos

En este caso solo hay dos columnas. Una de texto y la otra tipo Integer.

In [157]:
df_unfpa.sample(5)

,sdg
Textos_espanol,
"De hecho, la educación, medida por años de escolaridad, se correlaciona positivamente con la intensidad de capital en la manufactura (Qu y Cai, 2011). Dado que la mayor parte de la fuerza laboral en China está compuesta por graduados de secundaria y preparatoria, existe un amplio margen para invertir en educación y capacitación para impulsar la productividad laboral. La productividad laboral sería un 23% más alta si los graduados de la escuela secundaria básica tuvieran educación secundaria superior y se duplicaría si los graduados de la escuela secundaria superior tuvieran formación universitaria. Sin embargo, el logro de la educación superior por sí solo puede no ser suficiente. Para adaptarse a la estructura cambiante de la industria, se necesita un amplio conjunto de habilidades y amplios conocimientos generales, que faciliten la adquisición de nuevas habilidades.",4
"La resolución 1888 (2009) se centra en fortalecer el liderazgo, la experiencia y otras capacidades institucionales dentro de las Naciones Unidas y en los Estados miembros para ayudar a poner fin a la violencia sexual relacionada con los conflictos. En respuesta a la resolución 1888 (2009), el Secretario General nombró a un Representante Especial sobre la Violencia Sexual en los Conflictos. La resolución i960 (2010) también exige el establecimiento de arreglos de seguimiento, análisis y presentación de informes específicos para la violencia sexual relacionada con los conflictos. La resolución 2106 (2013) insta a todos los Estados miembros y entidades de las Naciones Unidas a hacer más para implementar los mandatos anteriores y afirma la centralidad de la igualdad de género y el empoderamiento político, social y económico de las mujeres para prevenir la violencia sexual en situaciones de conflicto armado y posteriores a conflictos.",5
"Este capítulo busca proporcionar contexto y perspectiva. Comienza analizando la demografía en Alemania y otros países de la OCDE, con un enfoque particular en la fertilidad, la composición familiar, el matrimonio y el aumento de la cohabitación. La sección 3 aborda el papel de la mujer en el mercado laboral. Encuentra que, aunque ha habido un fuerte crecimiento en el empleo femenino en los últimos 15 años, las mujeres alemanas continúan ganando menos que los hombres y con demasiada frecuencia están confinadas a trabajos a tiempo parcial.",5
"Está compuesto por ocho miembros, todos ellos altos representantes de la comunidad educativa internacional y tres analistas de la OCDE. Estos talleres también sirven para difundir mensajes clave y prácticas internacionales en México para fomentar una mayor reflexión y cambio. Este seminario de desarrollo de capacidades para formuladores de políticas mexicanos de alto nivel combinó un programa de capacitación activo con visitas a países a Chile (enero de 2010) para estudiar la política docente, a Canadá, Ontario (mayo de 2010) para estudiar liderazgo escolar y un módulo final sobre implementación. en México (junio de 2010).",4
"El aprendizaje es social y debe existir colaboración para desarrollar todas las habilidades que los estudiantes necesitan para el siglo XXI. En la pedagogía Lumiar, la voz del alumno es esencial para diseñar el itinerario de aprendizaje. En los niveles mÃ¡s profundos del aprendizaje, los estudiantes buscan y forman asociaciones para el desarrollo directo de ideas o la soluciÃ³n de problemas, se apropian de su aprendizaje tanto dentro como fuera de las aulas y contribuyen directamente al aprendizaje de los demÃ¡s, participan en el diseÃ±o, aplicaciÃ³n y mediciÃ³n de su propio aprendizaje y el de los demÃ¡s, y aprovechan y crean potentes tecnologÃas digitales que profundizan directamente en todos los aspectos del proceso de enseÃ±anza y aprendizaje. Los alumnos tienen voz y voto sobre la naturaleza de su producto final, aunque no sobre el método para desarrollarlo.",4


In [158]:
df_unfpa.dtypes

,0
sdg,int64


### Calidad y preparación de datos

1. Completitud

In [159]:
df_unfpa = df_unfpa.reset_index()

In [160]:
df_unfpa.isnull().sum()

,0
Textos_espanol,0
sdg,0


2. Unicidad / Duplicidad

In [161]:
df_unfpa.loc[df_unfpa.duplicated(subset = 'Textos_espanol', keep = False)]

,Textos_espanol,sdg


In [162]:
df_unfpa.duplicated(subset = 'Textos_espanol', keep = False).sum()

0

3. Consistencia

In [163]:
df_unfpa['sdg'].describe()

,sdg
count,4049.000000
mean,4.051124
std,0.814338
min,3.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [164]:
df_unfpa['sdg'].value_counts()

,count
sdg,
5,1451
4,1354
3,1244


4. Estandarización

Aquí vamos a proceder a estandarizar y procesar el texto. Primero, vamos a normalizar pasando todo a minúsculas y vamos a usar la técnica de lemmatización para pasar las palabras a su significado raíz. Igualmente, vamos a eliminar las stops words, que son las palabras que no aportan nada al análisis (como conectores). Por último, vamos a vectorizar el corpus ya procesado para poder aplicarle la técina de TF-IDF, la cual saca un coeficiente TF-IDF que mide tanto importancia como número de apariciones. De esta manera obtenemos una matriz (que convertimos nuevamente en dataframe) con las palabras y los coeficientes TF-IDF.

In [165]:
corpus = df_unfpa['Textos_espanol'].tolist()

In [166]:
# Descargar las stopwords en español
nltk.download('stopwords')
stop_words = stopwords.words('spanish')

# Tokenizador de NLTK y modelo de spaCy para lematización en español
wpt = WordPunctTokenizer()
nlp = spacy.load("es_core_news_sm")

# Función de normalización y lematización
def normalize_document(doc):
    # Convertir a minúsculas y eliminar espacios
    doc = doc.lower().strip()
    # Procesar todo el documento con spaCy de una sola vez
    spacy_doc = nlp(doc)
    # Lematización y eliminación de stopwords
    lemmatized_tokens = [token.lemma_ for token in spacy_doc if token.text not in stop_words]
    # Reconstruir el texto lematizado
    doc = ' '.join(lemmatized_tokens)
    return doc

# Vectorizar el corpus usando la función de normalización
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)
norm_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array(['ejemplo , nãºmero consulta externo especialista 319 cada mil derechohabient sp , comparaciã³n 338 620 cada mil derechohabient imss issste , respectivamente . si bien diferencia poder reflejar necesidad desigual ( poblaciã³n ligeramente mayor issste ) , poder justificar él manera . nãºmero receta poder ser surtir totalidad farmacã © utico debido falta existencia 33% dentro sp comparaciã³n 14% dentro imss segãºn dato encuesta ( aunque propio cifra instituto ss sugerir tasa mã¡s alto receta surtida ) . ambos cifra encontrar mã¡s alto ocde . gasto bolsillo reducir significativamente último década , pesar esfuerzo lograr cobertura sanitario universal través reforma sp .',
       '2007 , gobierno central financiar directamente solo 0,3% gasto total salud , realizar transferencia específico gasto salud ascender 5,6% gasto total . mayor parte gasto presupuestario salud realizar gobierno condado . , condado pobre solo poder ofrecer bajo nivel atención población local . gobierno provinci

In [167]:
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)
X = vectorizer.fit_transform(norm_corpus)  # X es la matriz TF-IDF resultante
print(X)
y = df_unfpa['sdg']
print(y)

  (0, 4105)	0.07354326654521462
  (0, 4767)	0.06748963753102628
  (0, 4823)	0.09368571135805155
  (0, 4316)	0.06927108970048174
  (0, 926)	0.08658303466763262
  (0, 2992)	0.07061380061314831
  (0, 1934)	0.07248555080510323
  (0, 3649)	0.08658303466763262
  (0, 1708)	0.08092014413972304
  (0, 4996)	0.07124349684105183
  (0, 4423)	0.09212843311909726
  (0, 4091)	0.06653518455772649
  (0, 726)	0.10938074450844637
  (0, 2324)	0.07223034828480408
  (0, 3400)	0.05202558477057706
  (0, 1832)	0.07053658112677096
  (0, 430)	0.08979406970573509
  (0, 418)	0.10619968192793425
  (0, 3246)	0.1676992201435787
  (0, 4639)	0.060597140816728415
  (0, 4566)	0.07827565594491989
  (0, 4514)	0.14175489827569973
  (0, 2704)	0.09693547789417266
  (0, 891)	0.1986972037719223
  (0, 3909)	0.07702850248740657
  :	:
  (4048, 1631)	0.139762127490604
  (4048, 3518)	0.12306544625133713
  (4048, 3705)	0.1121751265821091
  (4048, 690)	0.11592047718633405
  (4048, 2746)	0.10351650681746567
  (4048, 4840)	0.101904615594

In [168]:
# Obtener los nombres de las características (palabras) en el vocabulario
vocab = vectorizer.get_feature_names_out()

# Convertir la matriz dispersa TF-IDF a una matriz densa (completa)
X_dense = X.toarray()

# Crear un DataFrame de pandas con la matriz TF-IDF
df_tfidf = pd.DataFrame(X_dense, columns=vocab)

# Mostrar las primeras filas del DataFrame
print(df_tfidf.sample())

     000   01   02   03   06   07   08   09   10  100  ...  éxito  índice  \
659  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...    0.0     0.0   

     íntimo  óptimo  órden  órgano  último  únicamente  único  útil  
659     0.0     0.0    0.0     0.0     0.0         0.0    0.0   0.0  

[1 rows x 5000 columns]


### Conclusiones

Una vez teniendo procesado el texto, vamos a proceder a entrenar los algoritmos para llevar a cabo la tarea de clasificación. Los 3 algoritmos que vamos a usar para la clasificación son: Naive Bayes, KNN y Random Forest (?).

## Sección 3. Modelado y evaluación.

Separamos los datos en 20% test y 80% train.

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Naive Bayes (Andrés Romero)

In [170]:
# Entrenar el modelo Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_nb = nb_model.predict(X_test)

# Evaluar el rendimiento del modelo
print("Naive Bayes Classification Report:")
print(classification_report(y_test, y_pred_nb))

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           3       1.00      0.92      0.95       254
           4       0.97      0.97      0.97       265
           5       0.93      0.99      0.96       291

    accuracy                           0.96       810
   macro avg       0.97      0.96      0.96       810
weighted avg       0.96      0.96      0.96       810



## Regresión Logística (Nicolás Murillo)



In [171]:
 # se importa el modelo desde sklearn
from sklearn.linear_model import LogisticRegression

# EL siguiente código es quien entrena el modelo
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

#evaluación de los datos
y_pred_lr = lr_model.predict(X_test)

# impresión de los reportes del modelo
print("Logistic Regression Classification Report:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           3       0.98      0.97      0.98       254
           4       0.98      0.98      0.98       265
           5       0.97      0.99      0.98       291

    accuracy                           0.98       810
   macro avg       0.98      0.98      0.98       810
weighted avg       0.98      0.98      0.98       810



### Random Forest (Juan Pablo)

In [172]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Entrenar el modelo Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predicciones
y_pred_rf = clf.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.971604938271605
Classification Report:
               precision    recall  f1-score   support

           3       0.98      0.96      0.97       254
           4       0.98      0.97      0.98       265
           5       0.96      0.98      0.97       291

    accuracy                           0.97       810
   macro avg       0.97      0.97      0.97       810
weighted avg       0.97      0.97      0.97       810



## Sección 4. Resultados.

Finalmente el algoritmo que se escogió es el algoritmo de Regresion Logistica para la clasificación de las opiniones en los ODS dado que con este modelo se obtuvieron las mayores/mejores métricas para la clasificación. Los resultados muestran una exactitud del 97.16%, lo que significa que es este modelo es altamente preciso para clasificar las opiniones en las tres categorías.

**Métricas:**


In [173]:
classification_report(y_test, y_pred_lr)

'              precision    recall  f1-score   support\n\n           3       0.98      0.97      0.98       254\n           4       0.98      0.98      0.98       265\n           5       0.97      0.99      0.98       291\n\n    accuracy                           0.98       810\n   macro avg       0.98      0.98      0.98       810\nweighted avg       0.98      0.98      0.98       810\n'

**ODS 3 (Salud y Bienestar):**  
- **Precisión:** El modelo predijo correctamente el 98% de las veces cuando clasificó una opinión como relacionada con el ODS 3.
- **Recall:** El 96% de las opiniones realmente relacionadas con el ODS 3 fueron correctamente identificadas por el modelo.
- **F1-Score:** Esta métrica combina la precisión y el recall, indicando un rendimiento global muy alto del modelo en la predicción de opiniones sobre el ODS 3.  

**ODS 4 (Educación de Calidad):**  
- **Precisión:** El modelo identificó correctamente el 98% de las veces cuando clasificó una opinión como relacionada con el ODS 4.
- **Recall:** El 97% de las opiniones sobre el ODS 4 fueron correctamente detectadas.
- **F1Score-:** El modelo tiene un excelente rendimiento en la identificación de opiniones relacionadas con el ODS 4, combinando tanto la precisión como el recall.
  
**ODS 5 (Igualdad de Género):**  
- **Precisión:** El 96% de las veces, cuando el modelo predijo una opinión como relacionada con el ODS 5, fue correcta.
- **Recall:** El 98% de las opiniones sobre el ODS 5 fueron correctamente identificadas.
- **F1-Score:** El rendimiento del modelo en la clasificación de opiniones sobre el ODS 5 es igualmente alto, con un excelente equilibrio entre precisión y recall.

In [174]:
# Para cada ODS, obtener las palabras clave
for i, ods in enumerate([3, 4, 5]):
    print(f"Palabras clave para ODS {ods}:")

    # Filtrar las opiniones que el modelo clasificó correctamente como ODS actual
    indices = np.where(y_pred_lr == ods)[0]

    # Promedio de los valores TF-IDF de estas opiniones
    mean_tfidf = np.mean(X_test[indices].toarray(), axis=0)

    # Obtener las palabras con los valores TF-IDF más altos
    top_n = 10  # El número de palabras clave
    top_words_idx = mean_tfidf.argsort()[-top_n:][::-1]
    top_words = [vocab[idx] for idx in top_words_idx]
    print(top_words)

Palabras clave para ODS 3:
['salud', 'atención', 'médico', 'paciente', 'servicio', 'país', 'enfermedad', 'mental', 'sistema', 'sanitario']
Palabras clave para ODS 4:
['escuela', 'educación', 'estudiante', 'docente', 'educativo', 'escolar', 'alumno', 'aprendizaje', 'evaluación', 'ocde']
Palabras clave para ODS 5:
['mujer', 'género', 'hombre', 'poder', 'igualdad', 'trabajo', 'violencia', 'social', 'laboral', 'derecho']


### ODS 3

**Palabras clave:** salud, atención, médico, servicio, paciente, enfermedad, mental, sistema, sanitario.  
Las interpretaciones principales que podemos sacar de estas palabras clave son que las preocupaciones principales de los ciudadanos están relacionadas con la atención médica, el acceso a servicios médicos, y la salud mental.
Las palabras salud, ateción, servicio y médico nos pueden indicar que la atención médica es un tema de bastante importancia, por lo que corresponde a una de las principales preocupaciones. Seguidamente, palabras como enfermedad y mental nos indican la importancia que tienen las menciones de enfermedades mentales, lo que las convierte en otra de las principales preocupaciones. Por último, palabras como sistema y sanitario nos indican que el sistema de salud o sistema sanitario también es de alta importancia y, por ende, una de las principales preocupaciones de los ciudadanos.


**Posibles estrategias**  
Nuestro equipo, basándose en los resultados obtenidos, siguiere las siguientes estrategias que pueden ayudar con el problema y que puede ser importante que el negocio evalúe:
- Mejorar la infraestructura sanitaria
- Hacer campañas para facilitar el acceso al servicio de salud para todo el mundo y de manera gratuita de ser posible
- Hacer campañas relacionadas con la salud mental y la importancia de esta
- Facilitar el acceso a psicólogos y psiquiatras para todas las personas que lo necesiten, a un bajo costo o incluso gratis de ser necesario.

### ODS 4

**Palabras clave:** escuela, educación, estudiante, docente, educativo, escolar, alumno, aprendizaje, evaluación, ocde.

Las interpretaciones principales que podemos sacar de estas palabras clave son que las preocupaciones de los ciudadanos están centradas en la calidad de la educación, la infraestructura escolar, y el desempeño de los estudiantes y docentes.

Las palabras "educación", "escuela", y "educativo" nos indican que el sistema educativo y la infraestructura escolar son temas de gran importancia para los ciudadanos, probablemente relacionados con la calidad de la educación que reciben los estudiantes.

Las palabras "estudiante", "docente", y "alumno" reflejan un enfoque en el desempeño de los estudiantes y en el papel crucial de los docentes en la educación. Los ciudadanos muestran preocupación por la formación de los maestros y cómo esto afecta a los estudiantes.

Palabras como "aprendizaje" y "evaluación" sugieren que la preocupación no está solo en los resultados, sino también en los métodos de aprendizaje y cómo se evalúa a los estudiantes. Esto indica un interés en mejorar la calidad del aprendizaje, no solo en los resultados de las pruebas.

La mención de "OCDE" sugiere que existe una preocupación por cómo la educación local se compara con los estándares internacionales, y la gente parece estar interesada en elevar la calidad educativa para cumplir con esos estándares.

**Posibles estrategias**  
Nuestro equipo, basándose en los resultados obtenidos, sugiere las siguientes estrategias que pueden ayudar con los problemas identificados y que pueden ser importantes para que el negocio evalúe:

- Invertir en la mejora de la infraestructura escolar, asegurando que las instalaciones sean seguras, modernas y adecuadas para un entorno de aprendizaje óptimo. Es importante dar prioridad a las zonas rurales y desfavorecidas.

- Implementar programas de capacitación continua para los docentes, centrados en mejorar las metodologías de enseñanza y en fomentar un enfoque de aprendizaje más efectivo para los estudiantes.

- Rediseñar los sistemas de evaluación para que no solo midan el rendimiento académico, sino también el proceso de aprendizaje. Introducir evaluaciones más formativas y personalizadas para los estudiantes.

- Colaborar con organismos internacionales como la OCDE para adoptar mejores prácticas y estándares internacionales que aseguren una educación de calidad. Esto también puede incluir intercambios educativos y evaluaciones comparativas con otros países.

- Implementar programas que promuevan el aprendizaje personalizado, utilizando tecnología y enfoques innovadores que permitan a los estudiantes progresar a su propio ritmo y obtener una educación adaptada a sus necesidades.

### ODS 5

**Palabras clave:** mujer, género, hombre, poder, igualdad, trabajo, violencia, laboral, social, país.

Las interpretaciones principales que podemos sacar de estas palabras clave son que las preocupaciones de los ciudadanos están centradas en la igualdad de género, los problemas de violencia de género, y las desigualdades laborales entre hombres y mujeres.

Las palabras "mujer", "género" y "hombre" destacan que el tema de la discriminación de género es una preocupación central. La ciudadanía está preocupada por cómo los roles de poder entre hombres y mujeres afectan la igualdad de oportunidades.

Las palabras "igualdad" y "poder" sugieren que los ciudadanos están enfocados en la equidad de género, con especial atención en los desequilibrios de poder entre hombres y mujeres, tanto en el ámbito social como laboral.

Palabras como "trabajo", "laboral" y "violencia" indican que las desigualdades laborales y la violencia de género son temas clave. Estas preocupaciones reflejan un enfoque en la protección de los derechos de las mujeres y la necesidad de mejorar las condiciones laborales para ellas.

La palabra "social" sugiere que las preocupaciones no se limitan solo al entorno laboral, sino que también incluyen un contexto social más amplio, en el que las mujeres experimentan desigualdades en varias esferas de la vida.

**Posibles Estrategias**  
Nuestro equipo, basándose en los resultados obtenidos, sugiere las siguientes estrategias que pueden ayudar con los problemas identificados y que pueden ser importantes para que el negocio evalúe:

- Desarrollar e implementar campañas masivas de concientización sobre la violencia de género, enfocadas en educar a la sociedad sobre la prevención de la violencia y la importancia de crear entornos seguros para las mujeres.

- Impulsar políticas que promuevan la igualdad salarial y la eliminación de barreras laborales para las mujeres. Es fundamental implementar iniciativas que busquen cerrar la brecha de género en el acceso al trabajo y la promoción dentro de las empresas.

- Facilitar el acceso de las mujeres a programas de formación y emprendimiento que les permitan mejorar sus habilidades laborales y lograr un mayor empoderamiento económico. Esto puede incluir programas de capacitación digital o apoyo financiero para emprendedoras.

- Fomentar la participación de las mujeres en puestos de toma de decisiones en todos los niveles, tanto en el sector público como privado. Esto puede hacerse a través de iniciativas que promuevan la igualdad en el acceso a posiciones de liderazgo y poder.

- Iniciar campañas para desafiar los estereotipos de género arraigados en la sociedad y promover un entorno más equitativo en todas las esferas de la vida, incluyendo el hogar, la educación y los medios de comunicación.

### Por qué es útil esta información?

Este análisis es valioso para la organización porque las palabras clave extraídas reflejan las preocupaciones más comunes de los ciudadanos respecto a los ODS 3, 4 y 5. Esto permite a la organización enfocar sus esfuerzos en áreas prioritarias que realmente impactan a la población. Además, con estas palabras clave, la organización puede diseñar políticas públicas más efectivas y adaptadas a las necesidades expresadas. Por ejemplo, enfocarse en mejorar la atención médica, reforzar la educación de calidad y combatir la violencia de género. Por último, la información es útil para diseñar campañas de sensibilización dirigidas a los problemas específicos identificados, como la salud mental, la igualdad de género y la mejora educativa.

**Evaluación de impacto:** Estos resultados pueden ser una referencia para medir el impacto de las intervenciones futuras y ajustar las estrategias según las necesidades expresadas por la población.

### Datos de prueba

In [175]:
df_sin_etiquetar = pd.read_excel('/content/TestODScat_345.xlsx')
df_sin_etiquetar.shape

(702, 2)

In [176]:
df_sin_etiquetar.isnull().sum()

,0
Textos_espanol,0
sdg,0


In [177]:
corpus = df_sin_etiquetar['Textos_espanol'].tolist()

In [178]:
norm_corpus = normalize_corpus(corpus)

In [179]:
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words)
X = vectorizer.fit_transform(norm_corpus)  # X es la matriz TF-IDF resultante
X

<702x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 32730 stored elements in Compressed Sparse Row format>

In [180]:
# Obtener los nombres de las características (palabras) en el vocabulario
vocab = vectorizer.get_feature_names_out()

# Convertir la matriz dispersa TF-IDF a una matriz densa (completa)
X_dense = X.toarray()

# Crear un DataFrame de pandas con la matriz TF-IDF
df_tfidf = pd.DataFrame(X_dense, columns=vocab)

# Mostrar las primeras filas del DataFrame
print(df_tfidf.sample())

     000   02   03  048   05   07   08   09   10  100  ...  índice  índices  \
446  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...     0.0      0.0   

     íntimo  óptimo  órgano  último  únicamente     único  útil  œcooking  
446     0.0     0.0     0.0     0.0         0.0  0.113202   0.0       0.0  

[1 rows x 5000 columns]


In [181]:
# Hacer predicciones en los datos sin etiquetar
y_pred_nuevas_etiquetas = lr_model.predict(X)

In [182]:
# Agregar la columna con las etiquetas predichas
df_sin_etiquetar['sdg'] = y_pred_nuevas_etiquetas
# Guardar el DataFrame en un archivo Excel
df_sin_etiquetar.to_excel('TestODScat_345.xlsx', index=False)

## Sección 5. Mapa de actores relacionado con el producto de datos creado.

##Actor 1

Rol: Administrativo financiero de la UNFPA

Tipo: Financiador

Beneficio: El modelo le ayuda a ver cuales son los ods que necesitan mayor atención para asignar recursos eficientemente

Riesgo: Si el modelo es poco preciso puede asignar recursos incorrectamente



##Actor 2

Rol: Persona de la comunidad

Tipo: Beneficiado

Beneficio: El modelo identifica que a que ODS pertenece su problema en específico para que la UNFPA pueda atender precisamente sus necesidades y las de la comunidad.

Riesgo: Si el sistema no es preciso puede recibir recursos y beneficios que su comunidad no necesita

##Actor 3:

Rol: Administrador de datos o estadístico  

Tipo: Cliente

Beneficio: Recibe un software que automatiza su trabajo de clasificación

Riesgo: Puede generar reportes incorrectos o imprecisos a la UNFPA generados erróneamente por el modelo

## Sección 6. Trabajo en equipo

**Nicolás Murillo:**

Rol: Líder de negocio

Tareas realizadas: Sección 1 (entendimiento) , sección 5(mapa de actores) y algoritmo de regresión logística

Horas dedicadas: 5

Algoritmo asignado:Regresión logística

Retos en el proyecto: Falta de eficiencia en el manejo del tiempo y hacer las cosas con anticipación, errores en el código.

Puntos correspondientes: 33.3333/100

Puntos a mejorar: Realizar las cosas con más anticipación y sacar más provecho de las clases para aplicar los temas del curso al proyecto

**Andrés Romero:**

Rol: Líder de proyecto, datos y analítica

Tareas realizadas:   
- Analítica y procesamiento de datos
- Procesamiento de texto
- Modelo de Naive Bayes

Horas dedicadas: 6

Algoritmo asignado: Naive Bayes

Retos en el proyecto: Procesar el texto para poder pasarlo al modelo

Puntos correspondientes: 33.333.../100

Puntos a mejorar: Comunicación


**Juan Pablo Peña:**

Rol: Líder de análitica, Líder de proyecto

Tareas realizadas: Entendimiento de datos, análisis de los datos a trabajar y algoritmo Random Forest.

Horas dedicadas: 5

Algoritmo asignado:

Retos en el proyecto:

Puntos correspondientes: 33.333.../100

Puntos a mejorar: Distribución del tiempo
